# Task #1

## 1. Algoritmo Minimax en Juegos de Suma Cero

El algoritmo Minimax es un método de toma de decisiones utilizado en juegos de dos jugadores de suma cero con información perfecta (como el ajedrez o el to-ti-to). Su objetivo es encontrar la estrategia óptima para cada jugador suponiendo que ambos juegan de manera óptima.

### ¿Cómo funciona?

* Se representa el juego como un árbol de decisiones, donde los nodos representan estados del juego y las ramas representan movimientos posibles.
* Se asigna un valor a cada estado terminal (hoja), dependiendo de quién gana o pierde.
* El jugador MAX busca maximizar su ganancia, eligiendo el movimiento con el valor más alto.
* El jugador MIN (el oponente) busca minimizar la ganancia de MAX, eligiendo el movimiento con el valor más bajo.
* El valor minimax de un nodo es el valor que un jugador puede garantizarse si ambos juegan de manera óptima.
* El proceso se aplica recursivamente hasta llegar a la raíz del árbol, determinando el mejor movimiento inicial.

### Importancia del Valor Minimax

El valor minimax representa el mejor resultado garantizado para un jugador bajo el supuesto de que el oponente también juega óptimamente. Es clave en la toma de decisiones estratégicas en juegos competitivos.

## 2. Minimax vs. Poda Alfa-Beta

La poda alfa-beta es una optimización del algoritmo minimax que reduce la cantidad de nodos evaluados, mejorando la eficiencia sin afectar el resultado final.

### Diferencia Principal

Minimax examina todos los nodos del árbol de juego, mientras que la poda alfa-beta descarta ramas innecesarias, reduciendo el tiempo de ejecución.

### ¿Cómo funciona la poda alfa-beta?

* Usa dos valores:
    * Alfa (α): el mejor valor que MAX puede garantizar.
    * Beta (β): el mejor valor que MIN puede garantizar.
* Si en un nodo MIN se encuentra un valor menor que α, no se sigue explorando esa rama (porque MAX nunca la elegirá).
* Si en un nodo MAX se encuentra un valor mayor que β, se deja de explorar la rama (porque MIN nunca permitirá que se llegue ahí).

### Ejemplo de Complejidad

Supongamos un árbol de profundidad d y con b opciones por nivel:

* Minimax: O(b^d) (explora todos los nodos).
* Poda Alfa-Beta (con orden óptimo): O(b^(d/2)) (se exploran muchos menos nodos).

Esto significa que la poda alfa-beta reduce a la mitad la profundidad efectiva del árbol en el mejor caso.

## 3. Expectiminimax en Juegos con Incertidumbre

El expectiminimax es una extensión del algoritmo minimax para juegos donde hay elementos de azar, como los dados en el juego de Backgammon.

### Diferencias con Minimax

* Minimax asume que los jugadores toman decisiones deterministas y estratégicas.
* Expectiminimax incorpora nodos de azar que representan eventos probabilísticos, como tirar un dado o robar una carta.

### ¿Cómo funciona?

* Al igual que minimax, hay nodos MAX y MIN que representan jugadores.
* Se agregan nodos de azar donde se calculan valores esperados ponderados según las probabilidades de cada evento.
* En lugar de seleccionar el mejor o peor resultado, se toma la esperanza matemática de los posibles valores.

### Desafíos Claves

* Complejidad computacional: Al agregar nodos de azar, el árbol crece mucho más rápido.
* Estimación de probabilidades: Se requieren distribuciones de probabilidad precisas para hacer buenas decisiones.
* Juegos con información oculta: Como el póker, donde no conocemos todas las cartas del oponente, lo que hace más difícil calcular expectativas precisas.

In [2]:
import numpy as np
import random
import time

# Definir el tamaño del tablero
ROWS = 6
COLS = 7
PLAYER = 1  # Humano
AI = 2      # Inteligencia Artificial

# Crear tablero vacío
def create_board():
    return np.zeros((ROWS, COLS), dtype=int)

# Verificar si una columna es válida
def is_valid_move(board, col):
    return board[0, col] == 0

# Encontrar la siguiente fila vacía en una columna
def get_next_open_row(board, col):
    for row in range(ROWS - 1, -1, -1):
        if board[row, col] == 0:
            return row

# Colocar una ficha en el tablero
def drop_piece(board, row, col, piece):
    board[row, col] = piece

# Verificar si alguien ganó
def check_winner(board, piece):
    # Horizontal
    for row in range(ROWS):
        for col in range(COLS - 3):
            if all(board[row, col + i] == piece for i in range(4)):
                return True

    # Vertical
    for row in range(ROWS - 3):
        for col in range(COLS):
            if all(board[row + i, col] == piece for i in range(4)):
                return True

    # Diagonal positiva "\"
    for row in range(ROWS - 3):
        for col in range(COLS - 3):
            if all(board[row + i, col + i] == piece for i in range(4)):
                return True

    # Diagonal negativa "/"
    for row in range(3, ROWS):
        for col in range(COLS - 3):
            if all(board[row - i, col + i] == piece for i in range(4)):
                return True

    return False

# Evaluar el estado del tablero
def evaluate_window(window, piece):
    score = 0
    opp_piece = PLAYER if piece == AI else AI

    if window.count(piece) == 4:
        score += 100
    elif window.count(piece) == 3 and window.count(0) == 1:
        score += 5
    elif window.count(piece) == 2 and window.count(0) == 2:
        score += 2

    if window.count(opp_piece) == 3 and window.count(0) == 1:
        score -= 4

    return score

# Evaluar el tablero
def score_position(board, piece):
    score = 0

    # Centrar el juego
    center_array = [int(i) for i in list(board[:, COLS // 2])]
    score += center_array.count(piece) * 3

    # Evaluar horizontal
    for row in range(ROWS):
        row_array = [int(i) for i in list(board[row, :])]
        for col in range(COLS - 3):
            window = row_array[col:col + 4]
            score += evaluate_window(window, piece)

    # Evaluar vertical
    for col in range(COLS):
        col_array = [int(i) for i in list(board[:, col])]
        for row in range(ROWS - 3):
            window = col_array[row:row + 4]
            score += evaluate_window(window, piece)

    # Evaluar diagonales
    for row in range(ROWS - 3):
        for col in range(COLS - 3):
            window = [board[row + i, col + i] for i in range(4)]
            score += evaluate_window(window, piece)

    for row in range(3, ROWS):
        for col in range(COLS - 3):
            window = [board[row - i, col + i] for i in range(4)]
            score += evaluate_window(window, piece)

    return score

# Obtener las columnas válidas
def get_valid_moves(board):
    return [col for col in range(COLS) if is_valid_move(board, col)]

# Minimax con poda alpha-beta
def minimax(board, depth, alpha, beta, maximizingPlayer, pruning):
    valid_moves = get_valid_moves(board)
    is_terminal = check_winner(board, PLAYER) or check_winner(board, AI) or len(valid_moves) == 0

    if depth == 0 or is_terminal:
        if check_winner(board, AI):
            return (None, 100000)
        elif check_winner(board, PLAYER):
            return (None, -100000)
        else:
            return (None, score_position(board, AI))

    if maximizingPlayer:
        value = -np.inf
        best_col = random.choice(valid_moves)
        for col in valid_moves:
            temp_board = board.copy()
            row = get_next_open_row(temp_board, col)
            drop_piece(temp_board, row, col, AI)
            new_score = minimax(temp_board, depth - 1, alpha, beta, False, pruning)[1]
            if new_score > value:
                value = new_score
                best_col = col
            if pruning:
                alpha = max(alpha, value)
                if alpha >= beta:
                    break
        return best_col, value
    else:
        value = np.inf
        best_col = random.choice(valid_moves)
        for col in valid_moves:
            temp_board = board.copy()
            row = get_next_open_row(temp_board, col)
            drop_piece(temp_board, row, col, PLAYER)
            new_score = minimax(temp_board, depth - 1, alpha, beta, True, pruning)[1]
            if new_score < value:
                value = new_score
                best_col = col
            if pruning:
                beta = min(beta, value)
                if alpha >= beta:
                    break
        return best_col, value

# Menú principal
def main_menu():
    while True:
        print("\nConecta 4 - Selecciona una opción:")
        print("1. Jugar contra la IA")
        print("2. Simular IA vs IA")
        print("3. Salir")

        choice = input("Ingrese su opción: ")
        
        if choice == "1":
            play_game()
        elif choice == "2":
            play_ai_vs_ai()
        elif choice == "3":
            print("Saliendo del juego...")
            break
        else:
            print("Opción no válida, intenta de nuevo.")

# Juego Humano vs IA
def play_game():
    board = create_board()
    game_over = False

    print("\n¡Bienvenido a Conecta 4!")
    print(board)

    while not game_over:
        col = int(input("Elige una columna (0-6): "))
        if is_valid_move(board, col):
            row = get_next_open_row(board, col)
            drop_piece(board, row, col, PLAYER)
            print(board)
            if check_winner(board, PLAYER):
                print("¡Felicidades! Has ganado.")
                break

        print("\nTurno de la IA...")
        col, _ = minimax(board, 4, -np.inf, np.inf, True, pruning=True)
        row = get_next_open_row(board, col)
        drop_piece(board, row, col, AI)
        print(board)
        if check_winner(board, AI):
            print("La IA ha ganado.")
            break

# Simulación IA vs IA
def play_ai_vs_ai():
    print("\nSimulando IA vs IA...")
    play_game()

# Ejecutar menú
main_menu()



Conecta 4 - Selecciona una opción:
1. Jugar contra la IA
2. Simular IA vs IA
3. Salir

Simulando IA vs IA...

¡Bienvenido a Conecta 4!
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]]
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0]]

Turno de la IA...
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [1 0 0 2 0 0 0]]
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [1 0 1 2 0 0 0]]

Turno de la IA...
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 2 0 0 0 0]
 [1 0 1 2 0 0 0]]
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 2 1 0 0 0]
 [1 0 1 2 0 0 0]]

Turno de la IA...
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 2 0 0 0]
 [0 0 2 1 0 0 0]
 [1 0 1 2 0 0 0]]
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 2 0 0 0]
 [0 0 2 1 0 0 0]
 